### Import Required Libraries

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/YouTube-Project/data-youtube-comments.csv")

### Read the Data

In [ ]:
df.head()

,channel,name,comment,date,year,time,likes
0,Simplilearn,Simplilearn,🔥 Enroll for FREE Machine Learning Course &amp...,26-03-2021,2021,10:21:58,4
1,Simplilearn,Garde Tanmay,can I please get the datasets my email is tanm...,20-06-2021,2021,16:23:47,0
2,Simplilearn,Simplilearn,"Hi, thanks for watching our video. We have sen...",20-06-2021,2021,18:33:08,0
3,Simplilearn,Nirbhay Kumar,"Great video , May I request data set at reach2...",20-06-2021,2021,09:22:56,1
4,Simplilearn,Simplilearn,"Hi, thanks for watching our video. We have sen...",20-06-2021,2021,18:33:13,0


In [ ]:
df = df[50000:51000]

In [ ]:
df.head()

,channel,name,comment,date,year,time,likes
50000,deeplizard,deeplizard,I elaborate more on this in the corresponding ...,09-10-2019,2019,12:55:12,0
50001,deeplizard,Laeeq Khan,cooooooooooooooooooooooooooooool,03-10-2019,2019,06:51:34,1
50002,deeplizard,Michael Marzolf,I have seen many explanations -- yours is the ...,02-10-2019,2019,21:15:23,3
50003,deeplizard,Akshit Bhalla,Subscribed in 2min.,01-10-2019,2019,17:21:00,1
50004,deeplizard,Cheah Aun Mao,"Hi, may I know the function of different type ...",01-10-2019,2019,16:37:50,0


### EDA

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df['comment'][50000]

'I elaborate more on this in the corresponding blog in the section &quot;a note about the usage of the dot product&quot;:<br><a href="https://deeplizard.com/learn/video/YRhxdVk_sIs">https://deeplizard.com/learn/video/YRhxdVk_sIs</a>'

In [ ]:
df['comment'][50670]

'What&#39;s Python?'

### Split Text into Sentences

In [ ]:
from nltk.tokenize import sent_tokenize

sentences = []
for s in df['comment']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [ ]:
sentences[:10]

['I elaborate more on this in the corresponding blog in the section &quot;a note about the usage of the dot product&quot;:<br><a href="https://deeplizard.com/learn/video/YRhxdVk_sIs">https://deeplizard.com/learn/video/YRhxdVk_sIs</a>',
 'cooooooooooooooooooooooooooooool',
 'I have seen many explanations -- yours is the best yet -- thank you<br><br><br>mike',
 'Subscribed in 2min.',
 'Hi, may I know the function of different type of multiplier accumulator (MAC) unit contribute to accelerate the process of DSP?',
 'Perfect explanation as usual.',
 'Pictures not words ;).',
 'Nice!',
 '!',
 'Very helpful.']

### Download GloVe Word Embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2021-08-10 17:39:55--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-08-10 17:39:55--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-08-10 17:39:55--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)

400000

### Text Preprocessing

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

#### Import the stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#### Function to remove stopwords from our dataset

In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

### Vector Representation of Sentences

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

### Similarity Matrix Preparation

In [ ]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

#### Cosine Similarity to compute the similarity between a pair of sentences

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#### Initialize the matrix with cosine similarity scores

In [ ]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

### Applying PageRank Algorithm

In [ ]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

### Summary Extraction

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
# Extract top 10 sentences as the summary
for i in range(100):
  print(ranked_sentences[i][1])

Hey, it was a great video with really good explanation and use of really good examples....<br>Can I know the inspiration behind the channel&#39;s name?
Honestly, I believe that power point presentation of the code with graphic explanation like you did is far better than the actual video of a code running, and, us as &quot;students&quot; try to follow with the Instructor.
Hope I can continue to be a member for a long time.<br>Actually this is the first online course I have seen which does not throw syntax at you randomly and ACTUALLY tells you why you are doing something.
Since I see you message is new ...is had a question please ... once I found that this video is before 2 years is it still best to start with.
im not trying to hate but just saying if you dont practice the code he teaches on your own you will forget pretty much everything, dont do what i did i learnt the hard way make sure  to always practice code after a tutorial and try to do some projects after you learn a group of t